In [ ]:
#keras tuner를 사용하여 image classification application에 '하이퍼튜닝'(하이퍼파라미터)을 수행합니다.
#keras tuner는 tensorflow program에 대한 optimal hyper-parameter-set를 선택하는 데 도움을 주는 라이브러리입니다.

In [ ]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [ ]:
!pip install -q -U keras-tuner
import kerastuner as kt

In [ ]:
#download dataset
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
#Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [ ]:
#define model
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))

  #Tune the number of units in the first Dense layer
  #Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  #Tune the learning rate for the optimizer
  #Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

  return model

In [ ]:
#tuner를 instance화 하고 hyper-tuning 수행하기
#keras tuner에는 총 4가지 tuner가 있습니다. RandomSearch, Hyperband, BayesianOptimization, Sklearn
#여기서는 Hyperband tuner를 사용해 보겠다.
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', #최적화할 하이퍼모델
                     max_epochs = 10, #훈련할 최대 epoch 수
                     factor = 3,
                     directory = 'my_dir', #사용된 파라미터 저장할 폴더
                     project_name = 'intro_to_kt')

In [ ]:
#hyperparameter가 검색을 실행하기 전에 훈련 단계가 끝날 때마다 훈련 결과를 지우도록 콜백을 정의합니다.
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [24]:
#hyperparameter 검색을 실행합니다.
tuner.search(img_train, label_train, epochs=10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

#optimal hyper-parameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete.
The optimal number of units in the first densely-connected layer is {best_hps.get('units')} and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 22s]
val_accuracy: 0.8794999718666077

Best val_accuracy So Far: 0.8810999989509583
Total elapsed time: 00h 20m 12s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete.
The optimal number of units in the first densely-connected layer is 448 and the optimal learning rate for the optimizer is 0.001.



In [25]:
#optimal hyper-parameter로 model을 재훈련합니다.
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs=10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.4738 - accuracy: 0.8319 - val_loss: 0.4130 - val_accuracy: 0.8506
Epoch 2/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3606 - accuracy: 0.8701 - val_loss: 0.3913 - val_accuracy: 0.8563
Epoch 3/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3228 - accuracy: 0.8808 - val_loss: 0.3590 - val_accuracy: 0.8669
Epoch 4/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2988 - accuracy: 0.8895 - val_loss: 0.3640 - val_accuracy: 0.8702
Epoch 5/10
1875/1875 [==============================] - 10s 6ms/step - loss: 0.2804 - accuracy: 0.8962 - val_loss: 0.3374 - val_accuracy: 0.8816
Epoch 6/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2682 - accuracy: 0.9000 - val_loss: 0.3795 - val_accuracy: 0.8622
Epoch 7/10
1875/1875 [==============================] - 10s 6ms/step - loss: 0.2529 - accuracy: 0.9060 - val_loss: 0.3332 - val_ac